<a href="https://colab.research.google.com/github/vengie/neural-network/blob/main/Music%20Composition%20Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install music21

In [2]:
# !pip install keras

In [3]:
# !pip install tensorflow

In [4]:
# !pip install h5py

In [5]:
# !pip install glob2

In [6]:
import numpy as np
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense
from keras.utils import to_categorical
from music21 import converter, instrument, note, chord, stream
import glob

#Data Preparation

In [7]:
# # Upload MIDI files to the Colab environment
# uploaded = files.upload()

In [8]:
# # Load and preprocess the music dataset
# midi_files = glob.glob("*.mid")
# notes = []
# for file in midi_files:
#     midi = converter.parse(file)
#     for element in midi.flat:
#         if isinstance(element, note.Note):
#             notes.append(str(element.pitch))
#         elif isinstance(element, chord.Chord):
#             notes.append('.'.join(str(n) for n in element.normalOrder))

In [9]:
# Load and preprocess the music dataset
file_url='https://raw.githubusercontent.com/vengie/AML-3104--Neural-Networks-and-Deep-Learning/main/midi_songs/*.mid'
# midi_files = glob.glob("midi_songs/*.mid")
midi_files = glob.glob(file_url)
notes = []
for file in midi_files:
    midi = converter.parse(file)
    for element in midi.flat:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))

In [10]:
# Create input and output sequences
sequence_length = 100
pitch_names = sorted(set(notes))
note_to_int = dict((note, number) for number, note in enumerate(pitch_names))
network_input = []
network_output = []
for i in range(len(notes) - sequence_length):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])
n_patterns = len(network_input)
input_shape = (n_patterns, sequence_length, 1)
network_input = np.reshape(network_input, input_shape)
network_input = network_input / float(len(pitch_names))
network_output = to_categorical(network_output)

ValueError: ignored

In [ ]:
# Build the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=input_shape[1:], return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(128))
model.add(Dropout(0.3))
model.add(Dense(len(pitch_names), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
# Train the LSTM model
model.fit(network_input, network_output, epochs=50, batch_size=64)

In [ ]:
# Generate music using the trained model
start = np.random.randint(0, len(network_input)-1)
pattern = network_input[start]
prediction_output = []
for note_index in range(500):
    prediction_input = np.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(len(pitch_names))
    prediction = model.predict(prediction_input, verbose=0)
    index = np.argmax(prediction)
    result = pitch_names[index]
    prediction_output.append(result)
    pattern = np.append(pattern, index)
    pattern = pattern[1:len(pattern)]

In [ ]:
# Convert the output to musical notation
offset = 0
output_notes = []
for pattern in prediction_output:
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
    offset += 0.5

In [ ]:
# Create a MIDI file from the generated music
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='output.mid')